<a href="https://colab.research.google.com/github/RomaniiEkaterina/Phyton-cases/blob/main/Case1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2 кейс

**Соберите данные из Google Sheets и напишите функцию `generate_report`, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет отчет в файл `student_debt_report.txt`**

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [ ]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

--2025-06-22 11:17:09--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 140.82.113.4
Connecting to gist.github.com (gist.github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2025-06-22 11:17:10--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: ‘creds.json’

creds.json          100%[===================>]   2.30K  --.-KB/s    in 0s      

2025-06-22 11:17:10 (38.1 MB/s) - ‘creds.json’ saved [2

Чтобы посмотреть как выглядят сами гугл таблицы в оригинале - можете перейти по [ссылке](https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing) и изучить.

Небольшое описание столбцов в таблицах:  
**Лист1:** `student_id` - айди студента; `student_name` - имя студента; `installment` (Y/N) - факт наличия рассрочки у студента (Y - рассрочка есть, N - рассрочки нет).  
**Лист2:** `student_id` - айди студента; `last_payment_date` - дата последней полученной оплаты; `expected_payment_date` - дата, в которую ожидаем следующий платеж (рассчитывается от даты последнего платежа).  
**Лист3:** `student_id` - айди студента; `already_payed_amount` - уже выплаченная часть рассрочки; `left_to_pay` - оставшаяся (невыплаченная) часть; `one-time_payment` - единоразовый платеж; `installment_amount` - общая сумма, которая бралась в рассрочку.

Как примерно должен выглядеть результат:

In [ ]:
with open('result.txt') as f:
  print(f.read())

Студент Иванов У.У. - долг 100000 рублей
Студент Петров Е.Е. - долг 250000 рублей
Студент Сидоров И.И. - долг 10000 рублей


In [ ]:
#@title Если возникнут трудности с определением `scope`, подсказка:

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

**Примечание**

Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на **1 марта 2023 года**. А периоды внесения платежей для всех студентов одинаковы - **6 месяцев** (183 дня).

То есть, если ожидаемый платеж должен был пройти 3 февраля 2022 года, то к 1 марта 2023 студент просрочил уже три платежа (3 февраля 2022, 3 августа 2022 и 3 февраля 2023). При расчетах отталкивайтесь от даты ожидаемого платежа, разницу между платежами берите 183 дня.

**Решение**

Напишите свое решение ниже

In [ ]:
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import math
import csv

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)

client = gspread.authorize(creds)

spreadsheet = client.open_by_key("1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U")
worksheet1= spreadsheet.worksheet("Лист1")
sheet1 = worksheet1.get_all_records()

spreadsheet2 = client.open_by_key("1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U")
worksheet2= spreadsheet.worksheet("Лист2")
sheet2 = worksheet2.get_all_records()

spreadsheet3 = client.open_by_key("1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U")
worksheet3= spreadsheet.worksheet("Лист3")
sheet3 = worksheet3.get_all_records()
##values = worksheet.get_all_values()
def generate_report(sheet1, sheet2, sheet3):
  sheet2_dict = {str(row['student_id']): row for row in sheet2}
  sheet3_dict = {str(row['student_id']): row for row in sheet3}

  debt=[]

  for row1 in sheet1:
    student_id = str(row1['student_id'])
    if student_id in sheet2_dict and student_id in sheet3_dict:
      row2 = sheet2_dict.get(student_id, {})
      row3 = sheet3_dict.get(student_id, {})
      date_limit = datetime.strptime('01.03.2023', '%d.%m.%Y')
      if datetime.strptime(row2['last_payment_date'], '%d.%m.%Y') < date_limit:
        n=math.floor((date_limit-datetime.strptime(row2['expected_payment_date'], '%d.%m.%Y')).days/183)
        m=(n+1)*float(row3['one-time_payment'])
        if 0 < m <= (row3['left_to_pay']):
          debt.append({'student_id':student_id, 'student_name': row1.get('student_name',''), 'debt':m})
        elif m > (row3['left_to_pay']):
          debt.append({'student_id':student_id, 'student_name': row1.get('student_name',''), 'debt':row3['left_to_pay']})

  return debt

result= generate_report(sheet1, sheet2, sheet3)
output_file = 'combined_data.csv'

with open('student_debt_report.txt', 'w', encoding='utf-8') as f:
  for row in result:
    student_name=row['student_name']
    debt=int(float(row['debt']))
    f.write(f'Студент {student_name} - долг {debt} рублей\n')


with open('student_debt_report.txt', 'r', encoding='utf-8') as f:
    print(f.read())





Студент Кузнецова К.А. - долг 266666 рублей
Студент Петров К.А. - долг 66666 рублей
Студент Иванов А.А. - долг 100000 рублей
Студент Иванов П.П. - долг 116666 рублей
Студент Петров И.И. - долг 116666 рублей
Студент Петров А.А. - долг 16666 рублей
Студент Иванов Е.В. - долг 41666 рублей
Студент Николаева Е.В. - долг 100000 рублей
Студент Смирнова Е.В. - долг 233333 рублей
Студент Николаева М.А. - долг 33332 рублей
Студент Кузнецова А.А. - долг 133332 рублей
Студент Иванов А.А. - долг 25000 рублей
Студент Петров И.И. - долг 133332 рублей
Студент Кузнецова К.А. - долг 133333 рублей
Студент Кузнецова Е.В. - долг 66666 рублей
Студент Сидоров И.И. - долг 83333 рублей
Студент Петров И.И. - долг 16666 рублей
Студент Кузнецова И.И. - долг 233333 рублей
Студент Иванов А.А. - долг 266666 рублей
Студент Сидоров К.А. - долг 266666 рублей
Студент Смирнова И.И. - долг 200000 рублей
Студент Сидоров М.А. - долг 166666 рублей
Студент Сидоров Е.В. - долг 33333 рублей
Студент Иванов Е.В. - долг 100000 руб

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

import pandas as pd

user_answer = pd.read_csv('student_debt_report.txt')
correct_answer = pd.read_csv('student_debt.txt')



--2025-06-22 11:32:56--  https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.github.com (gist.github.com)... 140.82.112.4
Connecting to gist.github.com (gist.github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt [following]
--2025-06-22 11:32:56--  https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11325 (11K) [text/plain]
Saving to: ‘student_debt.txt.1’

student_debt.txt.1  100%[===================>]  11.06K  --.-KB/s    in 0.001s  

2025-06-22 11:32:56 (19.3 MB/

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
